In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
import torch
from torch import nn
from torchvision import datasets, transforms, utils
from torch.utils.data import Dataset, DataLoader

In [2]:
DATA_DIR =  "/content/drive/MyDrive/UTSA Student/Spring 2024/EE5423 HW4ML/Project/dataset"
KEY2DATADIR = {
    'MNIST-2': "./MNIST-2",
    "Fashion-MNIST-2": "./Fashion-MNIST-2",
    "Fashion-MNIST-3": "./Fashion-MNIST-3",
    "Syn-Dataset-4": "./Syn-Dataset-4",
    # "Syn-Dataset-16": "./Syn-Dataset-16"
}

KEY2FULLDIR = {
    'MNIST-2': os.path.join(DATA_DIR, KEY2DATADIR['MNIST-2']),
    'Fashion-MNIST-2': os.path.join(DATA_DIR, KEY2DATADIR['Fashion-MNIST-2']),
    'Fashion-MNIST-3': os.path.join(DATA_DIR, KEY2DATADIR['Fashion-MNIST-3']),
    'Syn-Dataset-4': os.path.join(DATA_DIR, KEY2DATADIR['Syn-Dataset-4']),
    # 'Syn-Dataset-16': os.path.join(DATA_DIR, KEY2DATADIR['Syn-Dataset-16'])
}

BATCH_SIZE = 4
EPOCHS = 50
DEVICE = 'cuda:0'
OPTIMIZER = torch.optim.Adam
LOSS_FN = torch.nn.CrossEntropyLoss()

In [3]:
def load_dataset(data_dir: str):
    # x_file = os.path.join(data_dir, './x.npy')
    # y_file = os.path.join(data_dir, './y.npy')
    x_train_file = os.path.join(data_dir, './x_train.npy')
    y_train_file = os.path.join(data_dir, './y_train.npy')
    x_test_file = os.path.join(data_dir, './x_test.npy')
    y_test_file = os.path.join(data_dir, './y_test.npy')

    # x = np.load(x_file)
    # y = np.load(y_file)
    x_train = np.load(x_train_file)
    y_train = np.load(y_train_file)
    x_test = np.load(x_test_file)
    y_test = np.load(y_test_file)

    return x_train, x_test, y_train, y_test

In [4]:
def create_simpleANN(input_dim, output_dim):
    simpleANN_model = nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Sigmoid()
    )
    simpleANN_model = simpleANN_model.to(DEVICE)
    return simpleANN_model

In [5]:
def create_complexANN(input_dim, output_dim):
    complexANN_model = nn.Sequential(
        nn.Linear(input_dim, 32),
        nn.ReLU(),
        nn.Linear(32, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, output_dim),
        nn.Sigmoid()
    )
    complexANN_model = complexANN_model.to(DEVICE)
    return complexANN_model

In [6]:
def train_one_epoch(model, train_dataloader, loss_fn, optimizer):
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair

        inputs, labels = data

        inputs = inputs.to(DEVICE)

        labels = labels.to(DEVICE)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs.to(torch.float32))

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

In [7]:
# Initializing in a separate cell so we can easily add more epochs to the same run
def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs):
    s_time = time.time()

    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []

    for epoch in range(epochs):
        print('EPOCH {}:'.format(epoch + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(model, train_dataloader, loss_fn, optimizer=optimizer)

        running_loss = 0.0
        running_tloss = 0.0
        train_total_correct = 0
        train_total_instances = 0
        test_total_correct = 0
        test_total_instances = 0
        # Set the model to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        model.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, data in enumerate(train_dataloader):
                # Every data instance is an input + label pair
                inputs, labels = data
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                outputs = model(inputs.to(torch.float32))
                classifications = torch.argmax(outputs, dim=1)
                correct_predictions = sum(classifications==labels).item()
                train_total_correct += correct_predictions
                train_total_instances += len(inputs)
                loss = loss_fn(outputs, labels)
                running_loss += loss

            for i, vdata in enumerate(test_dataloader):
                vinputs, vlabels = vdata
                vinputs = vinputs.to(DEVICE)
                vlabels = vlabels.to(DEVICE)
                voutputs = model(vinputs.to(torch.float32))
                classifications = torch.argmax(voutputs, dim=1)
                correct_predictions = sum(classifications==vlabels).item()
                test_total_correct += correct_predictions
                test_total_instances += len(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                running_tloss += vloss

        avg_loss = running_loss / (i + 1)
        avg_tloss = running_tloss / (i + 1)
        acc = train_total_correct/train_total_instances
        acc_test = test_total_correct/test_total_instances
        print("Train loss: {} Test loss {}".format(avg_loss, avg_tloss))
        print("Train acc: {} Test acc: {}".format(acc, acc_test))
        train_loss.append(avg_loss)
        test_loss.append(avg_tloss)
        train_acc.append(acc)
        test_acc.append(acc_test)

    train_time = time.time() - s_time
    return {
        'train_loss': train_loss,
        'test_loss': test_loss,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'train_time': train_time,
    }

In [8]:
REALCLASS2TRAINCLASSLABEL = {
    3: 0,
    6: 1,
    0: 2,
}

## MNIST-2

In [9]:
os.path.exists(KEY2FULLDIR['MNIST-2'])

True

In [10]:
DATA_NAME = "MNIST-2"
DATA_DIR = KEY2FULLDIR[DATA_NAME]

In [11]:
class MNIST2(Dataset):
    def __init__(self, data_dir, split=None, transform=None):
        x_train, x_test, y_train, y_test = load_dataset(data_dir)
        if split == 'train':
            self._X = x_train
            self._y = y_train
        elif split == 'test':
            self._X = x_test
            self._y = y_test
        elif split == None:
            exit(0)
        else:
            print("Wrong split !")
            exit(0)
        self._X = self._X.reshape(self._X.shape[0], self._X.shape[1]*self._X.shape[2])
        self._y = np.array([REALCLASS2TRAINCLASSLABEL[label] for label in self._y])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self._X[idx]/255.0, self._y[idx]

In [12]:
train_dataset = MNIST2(data_dir=DATA_DIR, split='train')
test_dataset = MNIST2(data_dir=DATA_DIR, split='test')

In [13]:
print('Training set has {} instances'.format(len(train_dataset)))
print('Validation set has {} instances'.format(len(test_dataset)))

Training set has 90 instances
Validation set has 10 instances


In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [15]:
input_dim = 4
output_dim = 2

### Simple ANN

In [16]:
simpleANN_model = create_simpleANN(input_dim, output_dim)

In [17]:
res = train(
        model=simpleANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(simpleANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 5.280436992645264 Test loss 0.6834701895713806
Train acc: 0.45555555555555555 Test acc: 0.4
EPOCH 2:
Train loss: 5.262364864349365 Test loss 0.6822509169578552
Train acc: 0.4666666666666667 Test acc: 0.4
EPOCH 3:
Train loss: 5.263825416564941 Test loss 0.6789376139640808
Train acc: 0.45555555555555555 Test acc: 0.4
EPOCH 4:
Train loss: 5.255505561828613 Test loss 0.6792353987693787
Train acc: 0.45555555555555555 Test acc: 0.4
EPOCH 5:
Train loss: 5.246719837188721 Test loss 0.6805461645126343
Train acc: 0.45555555555555555 Test acc: 0.4
EPOCH 6:
Train loss: 5.234959602355957 Test loss 0.6762840151786804
Train acc: 0.43333333333333335 Test acc: 0.4
EPOCH 7:
Train loss: 5.225302219390869 Test loss 0.6663785576820374
Train acc: 0.43333333333333335 Test acc: 0.4
EPOCH 8:
Train loss: 5.217953681945801 Test loss 0.6757616996765137
Train acc: 0.45555555555555555 Test acc: 0.4
EPOCH 9:
Train loss: 5.213662624359131 Test loss 0.6743606925010681
Train acc: 0.4888888888888889

In [18]:
mnist2_simpleANN_train_loss = res['train_loss'][-1]
mnist2_simpleANN_test_loss = res['test_loss'][-1]
mnist2_simpleANN_train_acc = res['train_acc'][-1]
mnist2_simpleANN_test_acc = res['test_acc'][-1]
mnist2_simpleANN_train_time = res['train_time']

In [19]:
print("Simple ANN's training score: {}".format(mnist2_simpleANN_train_acc))

Simple ANN's training score: 0.5444444444444444


In [20]:
print("Simple ANN's testing score: {}".format(mnist2_simpleANN_test_acc))

Simple ANN's testing score: 0.6


In [21]:
print("Simple ANN's training time: {} (seconds)".format(mnist2_simpleANN_train_time))


Simple ANN's training time: 2.8750178813934326 (seconds)


### Complex ANN

In [22]:
complexANN_model = create_complexANN(input_dim, output_dim)

In [23]:
res = train(
        model=complexANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(complexANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 5.295205593109131 Test loss 0.6934613585472107
Train acc: 0.5444444444444444 Test acc: 0.6
EPOCH 2:
Train loss: 5.268848419189453 Test loss 0.6839704513549805
Train acc: 0.5444444444444444 Test acc: 0.6
EPOCH 3:
Train loss: 5.232314109802246 Test loss 0.663013756275177
Train acc: 0.5444444444444444 Test acc: 0.6
EPOCH 4:
Train loss: 5.047208309173584 Test loss 0.652976393699646
Train acc: 0.5444444444444444 Test acc: 0.6
EPOCH 5:
Train loss: 4.749654769897461 Test loss 0.5340360999107361
Train acc: 0.5444444444444444 Test acc: 0.6
EPOCH 6:
Train loss: 4.549729347229004 Test loss 0.5275368690490723
Train acc: 0.7 Test acc: 0.9
EPOCH 7:
Train loss: 4.4609174728393555 Test loss 0.47513335943222046
Train acc: 0.7222222222222222 Test acc: 0.9
EPOCH 8:
Train loss: 4.38385009765625 Test loss 0.45152801275253296
Train acc: 0.7111111111111111 Test acc: 0.8
EPOCH 9:
Train loss: 4.362120628356934 Test loss 0.45562300086021423
Train acc: 0.7333333333333333 Test acc: 0.9
EPOCH 

In [24]:
mnist2_complexANN_train_loss = res['train_loss'][-1]
mnist2_complexANN_test_loss = res['test_loss'][-1]
mnist2_complexANN_train_acc = res['train_acc'][-1]
mnist2_complexANN_test_acc = res['test_acc'][-1]
mnist2_complexANN_train_time = res['train_time']

In [25]:
print("Complex ANN's training score: {}".format(mnist2_complexANN_train_acc))

Complex ANN's training score: 0.7666666666666667


In [26]:
print("Complex ANN's testing score: {}".format(mnist2_complexANN_test_acc))

Complex ANN's testing score: 0.9


In [27]:
print("Complex ANN's training time: {} (seconds)".format(mnist2_complexANN_train_time))

Complex ANN's training time: 4.1897993087768555 (seconds)


## Fashion-MNIST-2

In [28]:
DATA_NAME = "Fashion-MNIST-2"
os.path.exists(KEY2FULLDIR[DATA_NAME])
DATA_DIR = KEY2FULLDIR[DATA_NAME]

In [29]:
class FashionMNIST2(Dataset):
    def __init__(self, data_dir, split=None, transform=None):
        x_train, x_test, y_train, y_test = load_dataset(data_dir)
        if split == 'train':
            self._X = x_train
            self._y = y_train
        elif split == 'test':
            self._X = x_test
            self._y = y_test
        elif split == None:
            exit(0)
        else:
            print("Wrong split !")
            exit(0)
        self._X = self._X.reshape(self._X.shape[0], self._X.shape[1]*self._X.shape[2])
        self._y = np.array([REALCLASS2TRAINCLASSLABEL[label] for label in self._y])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self._X[idx]/255.0, self._y[idx]

In [30]:
train_dataset = FashionMNIST2(data_dir=DATA_DIR, split='train')
test_dataset = FashionMNIST2(data_dir=DATA_DIR, split='test')

In [31]:
print('Training set has {} instances'.format(len(train_dataset)))
print('Validation set has {} instances'.format(len(test_dataset)))

Training set has 90 instances
Validation set has 10 instances


In [32]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [33]:
input_dim = 4
output_dim = 2

### Simple ANN

In [34]:
simpleANN_model = create_simpleANN(input_dim, output_dim)

In [35]:
res = train(
        model=simpleANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(simpleANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 5.183048725128174 Test loss 0.6666265726089478
Train acc: 0.6666666666666666 Test acc: 0.7
EPOCH 2:
Train loss: 5.159914016723633 Test loss 0.6598657965660095
Train acc: 0.6777777777777778 Test acc: 0.7
EPOCH 3:
Train loss: 5.147794723510742 Test loss 0.6694318056106567
Train acc: 0.6888888888888889 Test acc: 0.7
EPOCH 4:
Train loss: 5.119607448577881 Test loss 0.6595859527587891
Train acc: 0.7 Test acc: 0.7
EPOCH 5:
Train loss: 5.1199188232421875 Test loss 0.6672658920288086
Train acc: 0.7222222222222222 Test acc: 0.7
EPOCH 6:
Train loss: 5.086686134338379 Test loss 0.661882758140564
Train acc: 0.7333333333333333 Test acc: 0.7
EPOCH 7:
Train loss: 5.080496788024902 Test loss 0.6604070663452148
Train acc: 0.7444444444444445 Test acc: 0.7
EPOCH 8:
Train loss: 5.050649642944336 Test loss 0.6624774932861328
Train acc: 0.7555555555555555 Test acc: 0.7
EPOCH 9:
Train loss: 5.040036201477051 Test loss 0.6444337368011475
Train acc: 0.7444444444444445 Test acc: 0.7
EPOCH 1

In [36]:
fashionmnist2_simpleANN_train_loss = res['train_loss'][-1]
fashionmnist2_simpleANN_test_loss = res['test_loss'][-1]
fashionmnist2_simpleANN_train_acc = res['train_acc'][-1]
fashionmnist2_simpleANN_test_acc = res['test_acc'][-1]
fashionmnist2_simpleANN_train_time = res['train_time']

In [37]:
print("Simple ANN's training score: {}".format(fashionmnist2_simpleANN_train_acc))

Simple ANN's training score: 0.7777777777777778


In [38]:
print("Simple ANN's testing score: {}".format(fashionmnist2_simpleANN_test_acc))

Simple ANN's testing score: 0.8


In [39]:
print("Simple ANN's training time: {} (seconds)".format(fashionmnist2_simpleANN_train_time))

Simple ANN's training time: 2.5208890438079834 (seconds)


### Complex ANN

In [40]:
complexANN_model = create_complexANN(input_dim, output_dim)

In [41]:
res = train(
        model=complexANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(complexANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 5.221521377563477 Test loss 0.68438720703125
Train acc: 0.5222222222222223 Test acc: 0.4
EPOCH 2:
Train loss: 4.9159417152404785 Test loss 0.6448922157287598
Train acc: 0.8 Test acc: 0.9
EPOCH 3:
Train loss: 4.304730415344238 Test loss 0.551845908164978
Train acc: 0.7888888888888889 Test acc: 0.9
EPOCH 4:
Train loss: 4.040945529937744 Test loss 0.5355964303016663
Train acc: 0.8111111111111111 Test acc: 0.7
EPOCH 5:
Train loss: 3.817584991455078 Test loss 0.47840481996536255
Train acc: 0.8 Test acc: 0.7
EPOCH 6:
Train loss: 3.7339696884155273 Test loss 0.4462679326534271
Train acc: 0.8333333333333334 Test acc: 0.7
EPOCH 7:
Train loss: 3.6893270015716553 Test loss 0.46086105704307556
Train acc: 0.8333333333333334 Test acc: 0.8
EPOCH 8:
Train loss: 3.570655345916748 Test loss 0.43562158942222595
Train acc: 0.8333333333333334 Test acc: 0.8
EPOCH 9:
Train loss: 3.518362045288086 Test loss 0.44840559363365173
Train acc: 0.8555555555555555 Test acc: 0.8
EPOCH 10:
Train lo

In [42]:
fashionmnist2_complexANN_train_loss = res['train_loss'][-1]
fashionmnist2_complexANN_test_loss = res['test_loss'][-1]
fashionmnist2_complexANN_train_acc = res['train_acc'][-1]
fashionmnist2_complexANN_test_acc = res['test_acc'][-1]
fashionmnist2_complexANN_train_time = res['train_time']

In [43]:
print("Complex ANN's training score: {}".format(fashionmnist2_complexANN_train_acc))

Complex ANN's training score: 0.9222222222222223


In [44]:
print("Complex ANN's testing score: {}".format(fashionmnist2_complexANN_test_acc))

Complex ANN's testing score: 0.9


In [45]:
print("Complex ANN's training time: {} (seconds)".format(fashionmnist2_complexANN_train_time))

Complex ANN's training time: 3.9199488162994385 (seconds)


## Fashion-MNIST-3

In [46]:
DATA_NAME = "Fashion-MNIST-3"
os.path.exists(KEY2FULLDIR[DATA_NAME])
DATA_DIR = KEY2FULLDIR[DATA_NAME]

In [47]:
class FashionMNIST3(Dataset):
    def __init__(self, data_dir, split=None, transform=None):
        x_train, x_test, y_train, y_test = load_dataset(data_dir)
        if split == 'train':
            self._X = x_train
            self._y = y_train
        elif split == 'test':
            self._X = x_test
            self._y = y_test
        elif split == None:
            exit(0)
        else:
            print("Wrong split !")
            exit(0)
        self._X = self._X.reshape(self._X.shape[0], self._X.shape[1]*self._X.shape[2])
        self._y = np.array([REALCLASS2TRAINCLASSLABEL[label] for label in self._y])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self._X[idx]/255.0, self._y[idx]

In [48]:
train_dataset = FashionMNIST3(data_dir=DATA_DIR, split='train')
test_dataset = FashionMNIST3(data_dir=DATA_DIR, split='test')

In [49]:
print('Training set has {} instances'.format(len(train_dataset)))
print('Validation set has {} instances'.format(len(test_dataset)))

Training set has 135 instances
Validation set has 15 instances


In [50]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [51]:
input_dim = 4
output_dim = 3

### Simple ANN

In [52]:
simpleANN_model = create_simpleANN(input_dim, output_dim)

In [53]:
res = train(
        model=simpleANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(simpleANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 9.556161880493164 Test loss 1.0453026294708252
Train acc: 0.31851851851851853 Test acc: 0.6
EPOCH 2:
Train loss: 9.528789520263672 Test loss 1.0451170206069946
Train acc: 0.31851851851851853 Test acc: 0.6
EPOCH 3:
Train loss: 9.501526832580566 Test loss 1.0480453968048096
Train acc: 0.32592592592592595 Test acc: 0.6
EPOCH 4:
Train loss: 9.474244117736816 Test loss 1.0505156517028809
Train acc: 0.34814814814814815 Test acc: 0.6
EPOCH 5:
Train loss: 9.447819709777832 Test loss 1.0522782802581787
Train acc: 0.35555555555555557 Test acc: 0.7333333333333333
EPOCH 6:
Train loss: 9.42262077331543 Test loss 1.054694414138794
Train acc: 0.4148148148148148 Test acc: 0.8
EPOCH 7:
Train loss: 9.400666236877441 Test loss 1.0567227602005005
Train acc: 0.45925925925925926 Test acc: 0.8
EPOCH 8:
Train loss: 9.370100021362305 Test loss 1.0612919330596924
Train acc: 0.4740740740740741 Test acc: 0.8
EPOCH 9:
Train loss: 9.346029281616211 Test loss 1.0631740093231201
Train acc: 0.4666

In [54]:
fashionmnist3_simpleANN_train_loss = res['train_loss'][-1]
fashionmnist3_simpleANN_test_loss = res['test_loss'][-1]
fashionmnist3_simpleANN_train_acc = res['train_acc'][-1]
fashionmnist3_simpleANN_test_acc = res['test_acc'][-1]
fashionmnist3_simpleANN_train_time = res['train_time']

In [55]:
print("Simple ANN's training score: {}".format(fashionmnist3_simpleANN_train_acc))

Simple ANN's training score: 0.4888888888888889


In [56]:
print("Simple ANN's testing score: {}".format(fashionmnist3_simpleANN_test_acc))

Simple ANN's testing score: 0.2


In [57]:
print("Simple ANN's training time: {} (seconds)".format(fashionmnist3_simpleANN_train_time))

Simple ANN's training time: 3.1010260581970215 (seconds)


### Complex ANN

In [58]:
complexANN_model = create_complexANN(input_dim, output_dim)

In [59]:
res = train(
        model=complexANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(complexANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 9.328341484069824 Test loss 1.0815023183822632
Train acc: 0.31851851851851853 Test acc: 0.6
EPOCH 2:
Train loss: 9.171720504760742 Test loss 1.0720044374465942
Train acc: 0.48148148148148145 Test acc: 0.7333333333333333
EPOCH 3:
Train loss: 8.357404708862305 Test loss 0.9938709735870361
Train acc: 0.5407407407407407 Test acc: 0.4
EPOCH 4:
Train loss: 8.055441856384277 Test loss 0.8644688129425049
Train acc: 0.5555555555555556 Test acc: 0.5333333333333333
EPOCH 5:
Train loss: 7.786763668060303 Test loss 0.8575284481048584
Train acc: 0.5555555555555556 Test acc: 0.5333333333333333
EPOCH 6:
Train loss: 7.755926132202148 Test loss 0.9045772552490234
Train acc: 0.5703703703703704 Test acc: 0.4666666666666667
EPOCH 7:
Train loss: 7.639382839202881 Test loss 0.8288436532020569
Train acc: 0.5851851851851851 Test acc: 0.5333333333333333
EPOCH 8:
Train loss: 7.598913192749023 Test loss 0.7961173057556152
Train acc: 0.5777777777777777 Test acc: 0.5333333333333333
EPOCH 9:
Tra

In [60]:
fashionmnist3_complexANN_train_loss = res['train_loss'][-1]
fashionmnist3_complexANN_test_loss = res['test_loss'][-1]
fashionmnist3_complexANN_train_acc = res['train_acc'][-1]
fashionmnist3_complexANN_test_acc = res['test_acc'][-1]
fashionmnist3_complexANN_train_time = res['train_time']

In [61]:
print("Complex ANN's training score: {}".format(fashionmnist3_complexANN_train_acc))

Complex ANN's training score: 0.6074074074074074


In [62]:
print("Complex ANN's testing score: {}".format(fashionmnist3_complexANN_test_acc))

Complex ANN's testing score: 0.4666666666666667


In [63]:
print("Complex ANN's training time: {} (seconds)".format(fashionmnist3_complexANN_train_time))

Complex ANN's training time: 5.562331438064575 (seconds)


## Synthetic-4

In [64]:
DATA_NAME = "Syn-Dataset-4"
print(os.path.exists(KEY2FULLDIR[DATA_NAME]))
DATA_DIR = KEY2FULLDIR[DATA_NAME]

True


In [65]:
class SynDataset4(Dataset):
    def __init__(self, data_dir, split=None, transform=None):
        x_train, x_test, y_train, y_test = load_dataset(data_dir)
        if split == 'train':
            self._X = x_train
            self._y = y_train
        elif split == 'test':
            self._X = x_test
            self._y = y_test
        elif split == None:
            exit(0)
        else:
            print("Wrong split !")
            exit(0)

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self._X[idx], self._y[idx]

In [66]:
train_dataset = SynDataset4(data_dir=DATA_DIR, split='train')
test_dataset = SynDataset4(data_dir=DATA_DIR, split='test')

In [67]:
print('Training set has {} instances'.format(len(train_dataset)))
print('Validation set has {} instances'.format(len(test_dataset)))

Training set has 90 instances
Validation set has 10 instances


In [68]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [69]:
input_dim = 4
output_dim = 2

### Simple ANN

In [70]:
simpleANN_model = create_simpleANN(input_dim, output_dim)

In [71]:
res = train(
        model=simpleANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(simpleANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 5.546730041503906 Test loss 0.739092230796814
Train acc: 0.4111111111111111 Test acc: 0.4
EPOCH 2:
Train loss: 5.55687141418457 Test loss 0.7236019372940063
Train acc: 0.4111111111111111 Test acc: 0.4
EPOCH 3:
Train loss: 5.542802810668945 Test loss 0.7167786955833435
Train acc: 0.4111111111111111 Test acc: 0.4
EPOCH 4:
Train loss: 5.509781837463379 Test loss 0.7578190565109253
Train acc: 0.4111111111111111 Test acc: 0.4
EPOCH 5:
Train loss: 5.5162787437438965 Test loss 0.728547215461731
Train acc: 0.4222222222222222 Test acc: 0.4
EPOCH 6:
Train loss: 5.486980438232422 Test loss 0.7651333808898926
Train acc: 0.43333333333333335 Test acc: 0.4
EPOCH 7:
Train loss: 5.468891143798828 Test loss 0.7307733297348022
Train acc: 0.43333333333333335 Test acc: 0.4
EPOCH 8:
Train loss: 5.445917129516602 Test loss 0.7604729533195496
Train acc: 0.43333333333333335 Test acc: 0.4
EPOCH 9:
Train loss: 5.455621719360352 Test loss 0.745980441570282
Train acc: 0.4444444444444444 Test a

In [72]:
synthetic4_simpleANN_train_loss = res['train_loss'][-1]
synthetic4_simpleANN_test_loss = res['test_loss'][-1]
synthetic4_simpleANN_train_acc = res['train_acc'][-1]
synthetic4_simpleANN_test_acc = res['test_acc'][-1]
synthetic4_simpleANN_train_time = res['train_time']

In [73]:
print("Simple ANN's training score: {}".format(synthetic4_simpleANN_train_acc))

Simple ANN's training score: 0.5666666666666667


In [74]:
print("Simple ANN's testing score: {}".format(synthetic4_simpleANN_test_acc))

Simple ANN's testing score: 0.4


In [75]:
print("Simple ANN's training time: {} (seconds)".format(synthetic4_simpleANN_train_time))

Simple ANN's training time: 2.098238468170166 (seconds)


### Complex ANN

In [76]:
complexANN_model = create_complexANN(input_dim, output_dim)

In [77]:
res = train(
        model=complexANN_model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=LOSS_FN,
        optimizer=OPTIMIZER(complexANN_model.parameters()),
        epochs=EPOCHS
)

EPOCH 1:
Train loss: 5.279750823974609 Test loss 0.7099796533584595
Train acc: 0.5222222222222223 Test acc: 0.3
EPOCH 2:
Train loss: 5.1786298751831055 Test loss 0.6929405927658081
Train acc: 0.5222222222222223 Test acc: 0.3
EPOCH 3:
Train loss: 4.7434492111206055 Test loss 0.6982836723327637
Train acc: 0.7333333333333333 Test acc: 0.5
EPOCH 4:
Train loss: 3.9709701538085938 Test loss 0.6664113998413086
Train acc: 0.8222222222222222 Test acc: 0.6
EPOCH 5:
Train loss: 3.400902509689331 Test loss 0.5094135999679565
Train acc: 0.9 Test acc: 0.8
EPOCH 6:
Train loss: 3.190520763397217 Test loss 0.4793729782104492
Train acc: 0.8888888888888888 Test acc: 0.8
EPOCH 7:
Train loss: 3.2742066383361816 Test loss 0.493020623922348
Train acc: 0.8888888888888888 Test acc: 0.8
EPOCH 8:
Train loss: 3.1882941722869873 Test loss 0.537598192691803
Train acc: 0.9111111111111111 Test acc: 0.7
EPOCH 9:
Train loss: 3.022024154663086 Test loss 0.547320544719696
Train acc: 0.9333333333333333 Test acc: 0.8
EPOCH

In [78]:
synthetic4_complexANN_train_loss = res['train_loss'][-1]
synthetic4_complexANN_test_loss = res['test_loss'][-1]
synthetic4_complexANN_train_acc = res['train_acc'][-1]
synthetic4_complexANN_test_acc = res['test_acc'][-1]
synthetic4_complexANN_train_time = res['train_time']

In [79]:
print("Complex ANN's training score: {}".format(synthetic4_complexANN_train_acc))

Complex ANN's training score: 0.9555555555555556


In [80]:
print("Complex ANN's testing score: {}".format(synthetic4_complexANN_test_acc))

Complex ANN's testing score: 0.8


In [81]:
print("Complex ANN's training time: {} (seconds)".format(synthetic4_complexANN_train_time))

Complex ANN's training time: 3.513808488845825 (seconds)


## Store the results

In [82]:
# prompt: print the results as data frames also include the running time with the columns as different dataset and rows as different metrics. Make sure the output follows each column is one dataset

import pandas as pd

# Create a dictionary to store the results
results = {
    "Dataset": ["MNIST-2", "Fashion-MNIST-2", "Fashion-MNIST-3", "Synthetic-4"],
    "Simple ANN Train Accuracy": [mnist2_simpleANN_train_acc, fashionmnist2_simpleANN_train_acc, fashionmnist3_simpleANN_train_acc, synthetic4_simpleANN_train_acc],
    "Simple ANN Test Accuracy": [mnist2_simpleANN_test_acc, fashionmnist2_simpleANN_test_acc, fashionmnist3_simpleANN_test_acc, synthetic4_simpleANN_test_acc],
    "Simple ANN Train Time": [mnist2_simpleANN_train_time, fashionmnist2_simpleANN_train_time, fashionmnist3_simpleANN_train_time, synthetic4_simpleANN_train_time],
    "Complex ANN Train Accuracy": [mnist2_complexANN_train_acc, fashionmnist2_complexANN_train_acc, fashionmnist3_complexANN_train_acc, synthetic4_complexANN_train_acc],
    "Complex ANN Test Accuracy": [mnist2_complexANN_test_acc, fashionmnist2_complexANN_test_acc, fashionmnist3_complexANN_test_acc, synthetic4_complexANN_test_acc],
    "Complex ANN Train Time": [mnist2_complexANN_train_time, fashionmnist2_complexANN_train_time, fashionmnist3_complexANN_train_time, synthetic4_complexANN_train_time],
}

# Create a DataFrame
df = pd.DataFrame(results)

# Print the DataFrame
print(df.to_string())


           Dataset  Simple ANN Train Accuracy  Simple ANN Test Accuracy  Simple ANN Train Time  Complex ANN Train Accuracy  Complex ANN Test Accuracy  Complex ANN Train Time
0          MNIST-2                   0.544444                       0.6               2.875018                    0.766667                   0.900000                4.189799
1  Fashion-MNIST-2                   0.777778                       0.8               2.520889                    0.922222                   0.900000                3.919949
2  Fashion-MNIST-3                   0.488889                       0.2               3.101026                    0.607407                   0.466667                5.562331
3      Synthetic-4                   0.566667                       0.4               2.098238                    0.955556                   0.800000                3.513808


In [83]:
datetime_str = datetime.now().strftime("%d-%m-%Y")
OUTPUT_DIR = "/content/drive/MyDrive/UTSA Student/Spring 2024/EE5423 HW4ML/Project/output/results"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
fulloutput_file = os.path.join(OUTPUT_DIR, "{}_ann_results.csv".format(datetime_str))
df.to_csv(fulloutput_file)